# Run matlab model in cantera
Runs the thermodynamic (shomate) parameters and the kinetic (arrhenius) parameters from the matlab output files in "../Grabow_matlab_data". 


to check:  
- do all Eas and As agree? 
- do rates agree when we convert? 

In [1]:
import pandas as pd
import numpy as np
import time
import cantera as ct
from matplotlib import pyplot as plt
import csv
import math
import os
import sys
from IPython.display import Image

import itertools
import logging
from collections import defaultdict

import scipy.linalg as la
import scipy.io 

print("Running Cantera version: {}".format(ct.__version__))

Running Cantera version: 2.6.0a1


### important constants or values 

In [2]:
site_density = 2.9e-09*(1e4)/(1e3) #(mol/cm^2) to kmol/m^2

mw_co = 28.01e-3  # [kg/mol]
mw_co2 = 44.01e-3  # [kg/mol]
mw_h2 = 2.016e-3  # [kg/mol]
mw_h2o = 18.01528e-3  # [kg/mol]

kb = 1.3806503E-23
Na = 6.0221415E23
mamu = 1.66053886E-27
h = 6.626068E-34
R = 8.314

eV2kJ = 96.485

### define functions to convert species and reactions

In [3]:
class matlab_species:
    def __init__(
         self,
         name,
         dHf0,
         S0,
         A,
         B,
         C,
         D,
         E,
         F,
         G,
         H,
         AtomicMass,
         EnergyOffset,
         BE_Fit,
         enthalpy,
         entropy,
         Pout,
         coverage,
    ): 
        """
        structure for storing species info from matlab file
        (shomates, etc)
        """
        self.dHf0 = dHf0
        self.S0 = S0
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        self.E = E
        self.F = F
        self.G = G
        self.H = H
        self.AtomicMass = AtomicMass
        self.EnergyOffset = EnergyOffset
        self.BE_Fit = BE_Fit
        self.enthalpy = enthalpy
        self.entropy = entropy
        self.Pout = Pout
        self.coverage = coverage

        # rename, remove g, replace "*"" with "X"
        if "g" in name:
            self.name = name.replace("g","")
        elif "*" in name:
            self.name = name.replace("*","X")
        else:
            self.name = name+"X"
            
        # hardcode rename of HCOOX to HCO2X because I am lazy
        if self.name == "HCO2X":
            self.name = "HCOOX"
        
        # calculate 7th parameter for cantera
        self.F_7param = self.dHf0 + self.F - self.H

        # add binding energy correction to F
        if abs(self.BE_Fit) > 0:
            self.F_7param = self.F_7param + self.BE_Fit*eV2kJ

        # add energy offset to F 
        if abs(self.EnergyOffset) > 0:
            self.F_7param = self.F_7param + self.EnergyOffset
        
        
        # create array of coefficients
        self.coeffs = [
            1000,
            self.A,
            self.B,
            self.C,
            self.D,
            self.E,
            self.F_7param,
            self.G,
            # for now, low and high are the same.
            self.A,
            self.B,
            self.C,
            self.D,
            self.E,
            self.F_7param,
            self.G,
        ]
            
        
        # create ct thermo object
        self.thermo = ct.ShomatePoly2(200, 2000, ct.one_atm, self.coeffs)

        

class matlab_rxn:
    
    def __init__(
         self,
         const_preexp,
         prettyprnt,
         rtype,
         species,
         stoichiometry,
         Ea,
         stickcoeff,
         omega,
         TSID,
         preexp,
         S_TS,
         Ha,
         ispecies,
         deltaH,
         dHDFT,
         EaDFT,
         deltaS,
         kfwd,
         Keq,
         krev,
         rfwd,
         rrev,
         rate,
    ): 
        """
        structure for storing reaction info from matlab file
        (deltaH, A-factor, etc.)
        """
        self.const_preexp = const_preexp
        self.prettyprnt = prettyprnt
        self.rtype = rtype
        self.species = species
        self.stoichiometry = stoichiometry
        self.Ea = Ea
        self.stickcoeff = stickcoeff
        self.omega = omega
        self.TSID = TSID
        self.preexp = preexp
        self.S_TS = S_TS
        self.Ha = Ha
        self.ispecies = ispecies
        self.deltaH = deltaH
        self.dHDFT = dHDFT
        self.EaDFT = EaDFT
        self.deltaS = deltaS
        self.kfwd = kfwd
        self.Keq = Keq
        self.krev = krev
        self.rfwd = rfwd
        self.rrev = rrev
        self.rate = rate
        
        
        # convert species list to new format
        new_spec = self.species
        for i , spec in enumerate(self.species):
            
            # rename, remove g, replace "*"" with "X"
            if "g" in spec:
                new_spec[i] = spec.replace("g","")
            elif "*" in spec:
                new_spec[i] = spec.replace("*","X")
            else:
                new_spec[i] = spec+"X"
                
            # hardcode rename of HCOOX to HCO2X because I am lazy
            if new_spec[i] == "HCO2X":
                new_spec[i]= "HCOOX"
        
        self.species = new_spec
        print(self.species)
        
        # put equation in same format as species
        self.new_equation = self.prettyprnt.replace(";","").replace("g", "")
        self.new_equation = self.new_equation.replace("*", "X").replace("->", "<=>")
        self.new_equation = self.new_equation.replace("HCO2X","HCOOX").replace(" ", "")
        
        # I am lazy and hardcoding these. cantera sometimes swaps 
        # reactants and products regardless of how they are in the input file
        if self.new_equation == "OHX+COX<=>COOHX+X":
            self.new_equation = "COX+OHX<=>COOHX+X"
            
        if self.new_equation == "H2OX+X<=>OHX+HX":
            self.new_equation = "H2OX+X<=>HX+OHX"
            
        if self.new_equation == "OHX+X<=>OX+HX":
            self.new_equation = "OHX+X<=>HX+OX"
            
        if self.new_equation == "OX+HCOX<=>OHX+COX":
            self.new_equation = "HCOX+OX<=>COX+OHX"
            
        if self.new_equation == 'OHX+HCOX<=>H2OX+COX':
            self.new_equation = 'HCOX+OHX<=>COX+H2OX'
            
        if self.new_equation == 'HCOOX+HCOX<=>HCOOHX+COX':
            self.new_equation = 'HCOOX+HCOX<=>COX+HCOOHX' 
            
        if self.new_equation == 'HCOOX+HCOX<=>H2CO2X+COX':
            self.new_equation = 'HCOOX+HCOX<=>COX+H2CO2X'
        
        if self.new_equation == "CH3OX+CH2OX<=>CH2OOCH3X+X":
            self.new_equation = 'CH2OX+CH3OX<=>CH2OOCH3X+X'
        
        # reaction rate is mol/site/sec
        # cantera A is m, Kmol, s units.
        if self.rtype == 0:
            # for now, don't convert
            A = self.stickcoeff
            self.A_orig = self.stickcoeff
        else:
            # for now, don't convert
            A = self.preexp 
            self.A_orig = self.preexp
        
        # no temperature exponents in model
        b = 0.0
        
        # Ea in kJ/mol, convert to J/kmol for cantera
        Ea = self.Ea*1e6
        
        # save Ea with no correction: 
        self.Ea_orig = self.Ea*1e6
        
        # make reaction object so we can update ct model
        self.arrhenius = ct.Arrhenius(A, b, Ea)


    def correct_rxns(self, species_dict, temp):
        """
        corrects reactions based on options chosen in matlab solver
        
        species_dict - dictionary of matlab_species objects, with the key as the species name
        temp - temp rxn is occurring at.
        """
        
        kB = 1.3806503E-23  # J/K
        h = 6.626068E-34;   # J
        R = 8.314           # kJ/mol/K?
        
        
        deltaS_TS = self.S_TS
        
        if not self.const_preexp and self.rtype > 0:
            for spec, spec_obj in species_dict.items():
                if spec in self.species:
                    
                    index = self.species.index(spec)
                    
                    if self.stoichiometry[index] < 0: #reactants only
                        deltaS_TS = deltaS_TS + self.stoichiometry[index]*spec_obj.S0
#                         print(spec, self.species, index,self.stoichiometry[index] )
#                         print(deltaS_TS)

                        
            self.preexp = kB*temp/h*np.exp(deltaS_TS/R)
            print("A-factor: ",self.preexp)
        
        # adjust Ea depending on omega
        self.Ea = self.EaDFT + self.omega*(self.deltaH - self.dHDFT)
        
        Ea = self.Ea*1e6
        
        # save corrected Ea
        self.Ea_corr = self.Ea*1e6
            
        if self.rtype == 0:
            A = self.stickcoeff*((R*temp)/1e-3)/site_density
        else:
            
            A = self.preexp/site_density
            
        # no temperature exponents in model
        b = 0.0
        
        # save corrected A
        self.A_corr = A
        
        # make reaction object so we can update ct model
        self.arrhenius = ct.Arrhenius(A, b, Ea)       

In [ ]:
#             if self.new_equation == "H2+2X<=>2HX":
#                 A = self.stickcoeff*((R*temp)/1e-3)/site_density
#                 print("found it")
#             else:
#                 A = self.stickcoeff*(R*temp)/1e-3

In [ ]:
# get feed number and run number from file name
new_run = False

if new_run: 
    path_str = "../Grabow_matlab_data/MeOH_Feed_1_Run_1.mat"
    file_name = path_str.split("/")[-1]
    print(file_name.split("_")[2])
    feed = int(file_name.split("_")[2])
    run = int(file_name.split("_")[4].split(".")[0])
else: 
    path_str = "../Grabow_matlab_data/original_runs/Graaf1988_ExpID_1.mat"


# matlab workspace with all relevant values
mat = scipy.io.loadmat(path_str)

species = mat['species']
reactions = mat['reaction']

if new_run:
    conditions = mat['condition']
    
else: 
    # typo in original grabow data
    conditions = mat['ccondition']

In [ ]:
# dictionary of matlab_species objects
species_dict = {}

for spec in species[0]:
    new_spec = matlab_species(
                name = str(spec[0][0]),
                dHf0 = float(spec[1][0]),
                S0= float(spec[2][0]),
                A = float(spec[3][0]),
                B = float(spec[4][0]),
                C = float(spec[5][0]),
                D = float(spec[6][0]),
                E = float(spec[7][0]),
                F = float(spec[8][0]),
                G = float(spec[9][0]),
                H = float(spec[10][0]),
                AtomicMass = float(spec[11][0]),
                EnergyOffset = 0 if spec[12][0].size==0 else float(spec[12][0]),
                BE_Fit= 0 if spec[13][0].size==0 else float(spec[13][0]),
                enthalpy = float(spec[14][0]),
                entropy = float(spec[15][0]),
                Pout = 0 if spec[16][0].size==0 else float(spec[16][0]),
                coverage = 0 if spec[17][0].size==0 else float(spec[17][0]),
    )
    species_dict[new_spec.name] = new_spec

In [ ]:
# dictionary of matlab_rxn objects
rxn_dict = {}

for rxn in reactions[0]:
    new_rxn = matlab_rxn(
        const_preexp = float(rxn[0][0]),
        prettyprnt = str(rxn[1][0]),
        rtype = float(rxn[2][0]),
        species = [str(i[0]) for i in rxn[3][0]],
        stoichiometry = list(rxn[4][0]),
        Ea = float(rxn[5][0]),
        stickcoeff = 0 if rxn[6][0].size ==0 else float(rxn[6][0]),
        omega = float(rxn[7][0]),
        TSID = 0 if rxn[8][0].size ==0 else float(rxn[8][0]),
        preexp = 0 if rxn[9][0].size ==0 else float(rxn[9][0]),
        S_TS = 0 if rxn[10][0].size ==0 else float(rxn[10][0]),
        Ha = 0 if rxn[11][0].size ==0 else float(rxn[11][0]),
        ispecies = rxn[12][0],
        deltaH = float(rxn[13][0]),
        dHDFT = float(rxn[14][0]),
        EaDFT = float(rxn[15][0]),
        deltaS = float(rxn[16][0]),
        kfwd = float(rxn[17][0]),
        Keq = float(rxn[18][0]),
        krev = float(rxn[19][0]),
        rfwd = float(rxn[20][0]),
        rrev = float(rxn[21][0]),
        rate = float(rxn[22][0]),
    )
    new_rxn.correct_rxns(species_dict, float(conditions['T']))
    rxn_dict[new_rxn.new_equation] = new_rxn

    


### now we have species_dict. make cantera species from that

### Thermo equation conversion (shomate 8 from matlab to shomate 7 in cantera)

#### Matlab: shomate 8

Source: NIST.  (2008). Water.  Retrieved  March  13,  2011,  from  NISTChemistry  Web  Book: 
http://webbook.nist.gov/cgi/cbook.cgi? ID=C7732185&Units=SI&Mask=1#Thermo-Gas

accessed from https://studylib.net/doc/9453117/shomate-equation

In [ ]:
Image(filename='../../External_data/images/Shomate_8.png', width = 700, height = 300)

#### Cantera: Shomate 7
source: cantera documentation https://cantera.org/science/science-species.html

In [ ]:
Image(filename='../../External_data/images/Shomate_7.png',width = 700, height = 300) 

### load temp, pressure, and mole fractions from matlab file

In [ ]:
if new_run:
    conditions = mat['condition']
    
else: 
    # typo in original grabow data
    conditions = mat['ccondition']

temp = float(conditions['T'])
pres = float(conditions['P'])

print(temp)

# order is COg, CO2g, H2g, CH3OHg, H2Og, HCOOHg, CH2Og
moles = conditions['molfrac'][0][0][0][0][1][0]

feed_moles = {"CO":moles[0], 
              "CO2":moles[1], 
              "H2":moles[2], 
              "CH3OH":moles[3], 
              "H2O":moles[4], 
              "HCOOH":moles[5], 
              "CH2O":moles[6],}

### load cti file that was imported directly from the supplementary data

In [ ]:
# import from supp data
file = "../../External_data/mech_grabow_new.cti"

gas_old = ct.Solution(file, "gas")
surf_old = ct.Interface(file,"surface1", [gas_old])

# initialize T and P
gas_old.TPX = temp, pres, feed_moles
surf_old.TP = temp, pres

### load cti file to a phase that we can modify to look like the matlab file.
this is basically a template that just happens to have all of the species and reactions we want, all the values will be overwritten

In [ ]:
file = "../../External_data/mech_grabow_new.cti"

gas = ct.Solution(file, "gas")
surf = ct.Interface(file,"surface1", [gas])

# initialize T and P
gas.TPX = temp, pres, feed_moles
surf.TP = temp, pres

### modify thermo so it matches matlab file

In [ ]:
for phase in (gas, surf):
    for S in phase.species():
        index = phase.species_index(S.name)
        print(f"old thermo {S.name} :\n{S.thermo.coeffs}")
        snew = species_dict[S.name].thermo
        S.thermo = snew
        phase.modify_species(index, S)
        print(f"new thermo {S.name} :\n{S.thermo.coeffs}")

### modify reactions so they match matlab file

In [ ]:
for rxn in surf.reactions():
    print(f"{rxn.equation} prior : {rxn.rate}")
    rxn.rate = rxn_dict[rxn.equation.replace(" ", "")].arrhenius
    print(f"{rxn.equation} after : {rxn.rate}")       

### compare enthalpy of reaction for each rxn

In [ ]:
print("{:30s}            {:7s}    {:7s}    {:7s} {:7s}  ".format("reaction","cantera","matlab","diff","supp data" ))
for index, rxn in enumerate(surf.reactions()):
    h_cantera = surf.delta_enthalpy[index]/1e6
    h_matlab = rxn_dict[rxn.equation.replace(" ", "")].deltaH
    h_diff = abs(h_cantera - h_matlab)
    
    h_supp_data = surf_old.delta_enthalpy[index]/1e6
    
    
    
    
    print(f"{rxn.equation:35s}:      {h_cantera:6.1f},  {h_matlab:6.1f},  {h_diff:6.1f}  {h_supp_data:6.1f}")
    

### compare species enthalpy

In [ ]:
gas.species("CH2O").thermo.coeffs

In [ ]:
gas_old.species("CH2O").thermo.coeffs

In [ ]:
for spec in gas.species():
    
    h_cantera = spec.thermo.h(gas.T)/1e6
    h_matlab = species_dict[spec.name].enthalpy
    
    h_diff = abs(h_cantera - h_matlab)
    h_diff_ev = (h_cantera - h_matlab) / 96 # 96 kJ to Ev 
    
    h_supp = gas_old.species(gas_old.species_index(spec.name)).thermo.h(gas.T)/1e6
    print(f"{spec.name:35s}:      {h_cantera:4.1e},  {h_matlab:4.1e},  {h_diff:4.1e} {h_diff_ev:4.2f} {h_supp:4.1e}")

In [ ]:
for spec in surf.species():
    
    h_cantera = spec.thermo.h(surf.T)/1e6
    h_matlab = species_dict[spec.name].enthalpy

    h_diff = abs(h_cantera - h_matlab)
    h_diff_ev = (h_cantera - h_matlab) / 96 # 96 kJ to Ev 
    
    h_supp = surf_old.species(surf_old.species_index(spec.name)).thermo.h(gas.T)/1e6
    
    print(f"{spec.name:35s}:      {h_cantera:4.1e},  {h_matlab:4.1e},  {h_diff:4.1e} {h_diff_ev:4.4f} {h_supp:4.1e}")

### compare species entropy

In [ ]:
for spec in gas.species():
    
    s_cantera = spec.thermo.s(gas.T)/1e3
    s_matlab = species_dict[spec.name].entropy
    
    s_diff = abs(s_cantera - s_matlab)
    print(f"{spec.name:35s}:      {s_cantera:4.1e},  {s_matlab:4.1e},  {s_diff:4.1e}")

In [ ]:
for spec in surf.species():
    s_cantera = spec.thermo.s(surf.T)/1e3
    s_matlab = species_dict[spec.name].entropy
    
    s_diff = abs(s_cantera - s_matlab)
    print(f"{spec.name:35s}:      {s_cantera:4.1e},  {s_matlab:4.1e},  {s_diff:4.1e}")

### compare rate constants
first, without converting

In [ ]:
print("{:30s}            {:7s}    {:7s}    {:7s}   {:7s} ".format("reaction","cantera","matlab","A/B", "supp data"))

for index, rxn in enumerate(surf.reactions()):
    k_cantera = surf.forward_rate_constants[index]
    k_matlab = rxn_dict[rxn.equation.replace(" ", "")].kfwd
    k_diff = abs(k_cantera/k_matlab)
    k_supp_data = surf_old.forward_rate_constants[index]
#     if k_diff <=100:
    print(f"{rxn.equation:35s}:      {k_cantera:4.2e}  {k_matlab:4.2e}  {k_diff:4.4f}   {k_supp_data :4.2e}")

### forward rate constant A factors $A_{fwd}$

In [ ]:
print("{:30s}            {:7s}    {:7s}    {:7s}    {:7s}  ".format("reaction","cantera","matlab","diff","supp data"))

for index, rxn in enumerate(surf.reactions()):
    A_cantera = surf.reactions()[index].rate.pre_exponential_factor
    A_matlab = rxn_dict[rxn.equation.replace(" ", "")].A_orig
    A_diff = A_cantera-A_matlab
    A_supp_data = surf_old.reactions()[index].rate.pre_exponential_factor
    
    if A_cantera !=0:
        print(f"{rxn.equation:35s}:      {A_cantera:4.2e},  {A_matlab:4.2e},  {A_diff:4.2f}   {A_supp_data:4.2e}")

### forward rate constant activation energy  $E_{a,fwd}$

In [ ]:
print("{:30s}            {:7s}    {:7s}    {:7s}    {:7s}  ".format("reaction","cantera","matlab","diff","supp data"))

for index, rxn in enumerate(surf.reactions()):
    Ea_cantera = surf.reactions()[index].rate.activation_energy
    Ea_matlab = rxn_dict[rxn.equation.replace(" ", "")].Ea_orig
    Ea_diff = Ea_cantera-Ea_matlab
    Ea_supp_data = surf_old.reactions()[index].rate.activation_energy
    
    if Ea_cantera !=0:
        print(f"{rxn.equation:35s}:      {Ea_cantera:4.2e},  {Ea_matlab:4.2e},  {Ea_diff:4.2f}   {Ea_supp_data:4.2e}")

### reverse rate constant $k_{rev}$

In [ ]:
print("{:30s}            {:7s}    {:7s}    {:7s}    ".format("reaction","cantera","matlab","diff"))
for index, rxn in enumerate(surf.reactions()):
    k_cantera = surf.reverse_rate_constants[index]/1e6
    k_matlab = rxn_dict[rxn.equation.replace(" ", "")].krev
    k_diff = abs(k_cantera - k_matlab)
    
#     if k_cantera !=0:
    print(f"{rxn.equation:35s}:      {k_cantera:4.1e},  {k_matlab:4.1e},  {k_diff:4.1e}")

### create a sbr model and run it 

In [ ]:
# create gas inlet
inlet = ct.Reservoir(gas)

# create gas outlet
exhaust = ct.Reservoir(gas)

# Reactor volume (divide by 2 per Graaf paper)
rradius = 35e-3
rlength = 70e-3
rvol = (rradius ** 2) * math.pi * rlength/2

# Catalyst Surface Area
site_density = (
    surf.site_density * 1000
)  # [mol/m^2]cantera uses kmol/m^2, convert to mol/m^2
cat_weight = float(conditions["CatalystWeight"])*1e-3 # [kg]
cat_site_per_wt = (300 * 1e-6) * 1000  # [mol/kg] 1e-6mol/micromole, 1000g/kg
cat_area = (cat_weight * cat_site_per_wt)/site_density # [m^3]

# reactor initialization
r = ct.IdealGasReactor(gas, energy="off")

# calculate the available catalyst area in a differential reactor
rsurf = ct.ReactorSurface(surf, r, A=cat_area)
r.volume = rvol
surf.coverages = "X:1.0"

# flow controllers (Graaf measured flow at 293.15 and 1 atm)
one_atm = ct.one_atm
FC_temp = 293.15
volume_flow = conditions["Flow"] * 60 /(100**3)  #cm^3/min to [m^3/s]
molar_flow = volume_flow * one_atm / (8.3145 * FC_temp)  # [mol/s]
mass_flow = molar_flow * (
    feed_moles["CO"] * mw_co + feed_moles["CO2"] * mw_co2 + feed_moles["H2"] * mw_h2 + feed_moles["H2O"] * mw_h2o
)  # [kg/s]
mfc = ct.MassFlowController(inlet, r, mdot=mass_flow)

# A PressureController has a baseline mass flow rate matching the 'master'
# MassFlowController, with an additional pressure-dependent term. By explicitly
# including the upstream mass flow rate, the pressure is kept constant without
# needing to use a large value for 'K', which can introduce undesired stiffness.
outlet_mfc = ct.PressureController(r, exhaust, master=mfc, K=0.01)

# initialize reactor network
sim = ct.ReactorNet([r])

In [ ]:
sim.step()

In [ ]:
for index, rxn in enumerate(surf.reactions()):
    print(surf.equilibrium_constants[index])

## rate calculation scratchpad

In [ ]:
# rates are off consistently for the following sticking coefficient reactions:
#                                           cantera   Matlab.   Cantera/matlab
# CO + X <=> COX                     :      5.30e+09  2.26e+08  23.4822   
# H2O + X <=> H2OX                   :      6.60e+09  2.81e+08  23.4823   
# HCOOH + X <=> HCOOHX               :      4.13e+09  1.76e+08  23.4822     
# CH3OH + X <=> CH3OHX               :      4.95e+09  2.11e+08  23.4821  

# why? changing the temp caused the value to change slightly, 
# but changing the pressure and number of sites had no effect. 
# plotted below to see if it was linear in T

In [ ]:
y = [516.7, 483.5, 499.3]
x = [24.3, 22.73, 23.482]

plt.scatter(x, y)

slope, intercept = np.polyfit(x, y, 1)
print(slope, intercept)

Here we calculate the cantera rate constant divided by the matlab one, by dividing thew the equation given in grabow's matlab file and the one given in the cantera documentation. 

## scratchpad
checking calcs above

In [ ]:
kb = 1.3806503E-23
Na = 6.0221415E23
mamu = 1.66053886E-27
h = 6.626068E-34
R = 8.314

dS_TS = 36.9904 + (-1)*72.6013

site_density = 2.9e-09*(1e4)/(1e3)

A = (kb * temp / h) * np.exp(dS_TS/R)
print("{:2.10e}".format(A))

A_calc = rxn_dict["H2OX+X<=>HX+OHX"].arrhenius.pre_exponential_factor
print("{:2.4e}".format(A_calc))

old_index = surf_old.reaction_equations().index("H2OX + X <=> HX + OHX")
A_old = surf_old.reactions()[old_index].rate.pre_exponential_factor
print("{:2.4e}".format(A_old))

print(site_density)

### activation energy: 

In [ ]:
kb = 1.3806503E-23
Na = 6.0221415E23
mamu = 1.66053886E-27
h = 6.626068E-34
R = 8.314

dS_TS = 36.9904 + (-1)*72.6013

Ea  = rxn_dict["H2OX+X<=>HX+OHX"].EaDFT + rxn_dict["H2OX+X<=>HX+OHX"].omega*(rxn_dict["H2OX+X<=>HX+OHX"].deltaH - rxn_dict["H2OX+X<=>HX+OHX"].dHDFT)
print("{:2.10e}".format(Ea))

Ea_calc = rxn_dict["H2OX+X<=>HX+OHX"].arrhenius.activation_energy/1e6
print("{:2.10e}".format(Ea_calc))

old_index = surf_old.reaction_equations().index("H2OX + X <=> HX + OHX")
Ea_old = surf_old.reactions()[old_index].rate.activation_energy/1e6
print("{:2.4e}".format(Ea_old))


## compare rates
get last row of concentrations to check rates

In [ ]:
# load array of partial pressures. load species table to steal header line from that.
partial_press = mat['Y']
time = mat["T"]
pres = float(mat["ccondition"]["P"])
temp = float(mat["ccondition"]["T"])

In [ ]:
# load reactions to dataframe 
reactions_df = pd.DataFrame(data = reactions[0])

In [ ]:
# Extract species names, remove unnecessary characters from column headers
species_names = []
translation = {39: None, 91: None, 93: None} # remove ', [, and ]

for spec in range(len(species[0,:])):
    species_string = str(species[0,spec][0])
    species_string = species_string.translate(translation)
    species_names.append(species_string)
    
# create data frame with species names as column headers
species_df = pd.DataFrame(data=partial_press,columns=species_names)

In [ ]:
species_df.tail(2).head(1)

not ideal, but get second to last row of dataframe with tail and head 

In [ ]:
concentration_dict = species_df.tail(2).head(1).to_dict(orient='list')

In [ ]:
concentration_dict_gas = {}
concentration_dict_surf = {}

# go through each species and rename. 
# go through each activity for gas species, convert to mole fraction. 

for spec, activ in concentration_dict.items(): 
    
    # rename, remove g, replace "*"" with "X"
    if "g" in spec:
        mole_frac = float(activ[0])/pres
        new_spec = spec.replace("g","")
        concentration_dict_gas[new_spec] = mole_frac
        
    elif "*" in spec:
        new_spec = spec.replace("*","X")
        mole_frac = float(activ[0])
        concentration_dict_surf[new_spec] = mole_frac
        
    else:
        new_spec = spec+"X"
        
        # hardcode rename of HCOOX to HCO2X because I am lazy
        if new_spec == "HCO2X":
            new_spec= "HCOOX"
        
        mole_frac = float(activ[0])
        concentration_dict_surf[new_spec] = mole_frac




In [ ]:
concentration_dict_surf

## initialize reactor with second to last row of mole fractions
step forward and get rate constants

### first, do a single calculation from matlab data, and see if this is how the rates were calculated

do it for CO+X<=>COX

turns out they were pretty close. 

In [ ]:
k_fwd = rxn_dict['CO+X<=>COX'].kfwd

CO_P = concentration_dict["COg"][0] # CO partial pressure
X_C = concentration_dict["*"][0]  # X surface fraction

rate_rxn1 = k_fwd*CO_P*X_C 
rate_rxn_1_matlab = rxn_dict['CO+X<=>COX'].rfwd
print("our calc: {:1.4e}".format(rate_rxn1))
print("matlab value: {:1.4e}".format(rate_rxn_1_matlab))

In [ ]:
# get length of last timestep
delta_T = float(time[-1]) - float(time[-2])

In [ ]:
# initialize T and P
gas.TPX = temp, pres, concentration_dict_gas
surf.TP = temp, pres
surf.coverages = concentration_dict_surf

# create gas inlet
inlet = ct.Reservoir(gas)

# create gas outlet
exhaust = ct.Reservoir(gas)

# Reactor volume (divide by 2 per Graaf paper)
rradius = 35e-3
rlength = 70e-3
rvol = (rradius ** 2) * math.pi * rlength/2


site_density = (
    surf.site_density * 1000
)  # [mol/m^2]cantera uses kmol/m^2, convert to mol/m^2
cat_weight = float(conditions["CatalystWeight"])*1e-3 # [kg]
cat_site_per_wt = 5*61.67*1e-6*1e3 # [mol/kg] 1e-6mol/micromole, 1000g/kg
cat_area = (cat_weight * cat_site_per_wt)/site_density  # [m^3]


# reactor initialization
r = ct.IdealGasReactor(gas, energy="off")

# calculate the available catalyst area in a differential reactor
rsurf = ct.ReactorSurface(surf, r, A=cat_area)
r.volume = rvol
surf.coverages = "X:1.0"

# flow controllers (Graaf measured flow at 293.15 and 1 atm)
one_atm = ct.one_atm
FC_temp = 293.15
volume_flow = conditions["Flow"] * 60 /(100**3)  #cm^3/min to [m^3/s]
molar_flow = volume_flow * one_atm / (8.3145 * FC_temp)  # [mol/s]
mass_flow = molar_flow * (
    feed_moles["CO"] * mw_co + feed_moles["CO2"] * mw_co2 + feed_moles["H2"] * mw_h2 + feed_moles["H2O"] * mw_h2o
)  # [kg/s]
mfc = ct.MassFlowController(inlet, r, mdot=mass_flow)

# A PressureController has a baseline mass flow rate matching the 'master'
# MassFlowController, with an additional pressure-dependent term. By explicitly
# including the upstream mass flow rate, the pressure is kept constant without
# needing to use a large value for 'K', which can introduce undesired stiffness.
outlet_mfc = ct.PressureController(r, exhaust, master=mfc, K=0.01)

# initialize reactor network
sim = ct.ReactorNet([r])

## advance sim by $\Delta$T

In [ ]:
sim.advance(delta_T)

# match rates

matlab rate units: $\frac{mol}{site*s}$  

cantera rate units: $\frac{kmol}{m^{2}*s}$  

multiplication factor: $\frac{kmol}{m^{2}*s}*\frac{10^{3}mol}{kmol}*\frac{m^{2}}{2.9*10^{-5}mol_{site}}*\frac{mole}{6.02*10^{23} sites}$


In [ ]:
mult_factor = 1e3/(2.9e-5*6.02e23)
mult_factor

In [ ]:
print("{:30s}            {:7s}    {:7s}    {:7s}  ".format("reaction","cantera","matlab","diff",))

for index, rxn in enumerate(surf.reactions()):
    r_cantera = surf.forward_rates_of_progress[index]
    r_matlab = rxn_dict[rxn.equation.replace(" ", "")].rfwd
    r_diff = r_cantera/r_matlab
    
    if r_cantera !=0:
        print(f"{rxn.equation:35s}:      {r_cantera:4.4e},  {r_matlab:4.4e},  {r_diff:4.4e}")